In [17]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import requests
from PyPDF2 import PdfReader
import os
from docx import Document

def setup_driver():  
    """Configure and return a headless Chrome driver"""  
    options = webdriver.ChromeOptions()  
    options.add_argument("--headless")  
    options.add_argument("--disable-gpu")  
    options.add_argument("--no-sandbox")  
    options.add_argument("--disable-dev-shm-usage")  
    options.add_argument("--window-size=1920x1080")  
    
    return webdriver.Chrome(options=options)  

driver = setup_driver()

In [1]:
years = ['2025', '2024', '2023', '2022', '2021', '2020', '2019', '2018', '2017' , '2016',  '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004' ,'2003', '2002', '2001', '2000', '1999', '1998' ,'1997', '1996', '1995', '1994', '1993', '1992', '1991', '1990', '1989', '1988', '1987', '1986', '1985', '1984', '1983', '1982', '1981', '1980', '1979', '1978', '1977', '1976' ,'1975', '1974', '1972', '1971', '1970', '1969' , '1967', '1966', '1964', '1961' ,'1954', '1944', '1938', '1930']

url = 'https://new.kenyalaw.org/judgments/all/2025/'

pager_url = 'https://new.kenyalaw.org/judgments/all/2025/?page=2'
data = []

def get_page(url):
    driver.get(url)
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="doc-table"]/table/tbody/tr')))
        articles = driver.find_elements(By.XPATH,'//*[@id="doc-table"]/table/tbody/tr')
        # print(len(articles))

        for article in articles:
            try:
                
                article_title = article.find_element(By.XPATH, './/td')
                article_title = article_title.text

                # print(article_title)
                print('-----------------------')
                article_link = article.find_element(By.XPATH, './/td/a').get_attribute('href')
                # print(article_link)
                content = get_content(article_link)


                data.append({
                    'title': article_title,
                    'link': article_link,
                    'content': content
                })
                print(data)
                df = pd.DataFrame(data)
                df.to_csv('judgements_pg0.csv', index=False)

            except NoSuchElementException:
                print('No link found in this element')               
                
                # print(article.get_attribute('innerHTML'))
                # print('-----------------------')

    except TimeoutException:
        print("TimeoutException: Element not found")


def download_docx(docx_url, output_path):
    response = requests.get(docx_url)
    with open(output_path, 'wb') as file:
        file.write(response.content)
    print(f"DOCX downloaded to {output_path}")

def extract_docx_content(docx_path):
    content = ""
    doc = Document(docx_path)
    for para in doc.paragraphs:
        content += para.text + "\n"
    return content

def get_content(url):
    driver.get(url)
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'document-content__inner')))
        docx_element = driver.find_element(By.XPATH, '//a[contains(text(), "Download DOCX")]')
        docx_url = docx_element.get_attribute('href')
        
        # Download the DOCX
        docx_path = 'downloaded_document.docx'
        download_docx(docx_url, docx_path)
        
        # Extract content from the downloaded DOCX
        docx_content = extract_docx_content(docx_path)
        print(docx_content)
        
        # Optionally, delete the downloaded DOCX after extraction
        os.remove(docx_path)
        
    except TimeoutException:
        print("TimeoutException: Element not found")


# get_page(url)
# get_content("https://new.kenyalaw.org/akn/ke/judgment/kemc/2025/30/eng@2025-03-10")


In [ ]:
import time
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from docx import Document
from datetime import datetime

def setup_driver():
    """Configure and return a headless Chrome driver"""
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--window-size=1920x1080")
    
    return webdriver.Chrome(options=options)

driver = setup_driver()

years = ['2025', '2024', '2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000', '1999', '1998', '1997', '1996', '1995', '1994', '1993', '1992', '1991', '1990', '1989', '1988', '1987', '1986', '1985', '1984', '1983', '1982', '1981', '1980', '1979', '1978', '1977', '1976', '1975', '1974', '1972', '1971', '1970', '1969', '1967', '1966', '1964', '1961', '1954', '1944', '1938', '1930']


pager_url = 'https://new.kenyalaw.org/judgments/all/2025/?page=2'
data = []

def get_page(url):
    driver.get(url)
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="doc-table"]/table/tbody/tr')))
        articles = driver.find_elements(By.XPATH, '//*[@id="doc-table"]/table/tbody/tr')
        # print(len(articles))

        for article in articles:
            try:
                article_title = article.find_element(By.XPATH, './/td[2]').text
                article_link = article.find_element(By.XPATH, './/td[2]/a').get_attribute('href')
                print(f"Title: {article_title}")
                print(f"Link: {article_link}")
                print('-----------------------')
                
                content = get_content(article_link)

                data.append({
                    'title': article_title,
                    'link': article_link,
                    'content': content
                })
                print(data)
                df = pd.DataFrame(data)
                df.to_csv('judgements_pg1.csv', index=False)

            except NoSuchElementException:
                print('No link found in this element')               

    except TimeoutException:
        print("TimeoutException: Element not found")

def download_docx(docx_url, output_path):
    response = requests.get(docx_url)
    with open(output_path, 'wb') as file:
        file.write(response.content)
    print(f"DOCX downloaded to {output_path}")

def extract_docx_content(docx_path):
    content = ""
    doc = Document(docx_path)
    for para in doc.paragraphs:
        content += para.text + "\n"
    return content

def get_content(url):
    driver.get(url)
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'document-content__inner')))
        docx_element = driver.find_element(By.XPATH, '//a[contains(text(), "Download DOCX")]')
        docx_url = docx_element.get_attribute('href')
        
        # Generate a unique filename using a timestamp
        timestamp = datetime.now().strftime('%Y%m%d%H%M%S%f')
        docx_path = f'downloaded_document_{timestamp}.docx'
        
        # Download the DOCX
        download_docx(docx_url, docx_path)
        
        # Extract content from the downloaded DOCX
        docx_content = extract_docx_content(docx_path)
        print(docx_content)
        
        # Optionally, delete the downloaded DOCX after extraction
        os.remove(docx_path)
        
        return docx_content
        
    except TimeoutException:
        print("TimeoutException: Element not found")
        return ""
    except NoSuchElementException:
        print("No DOCX link found")
        return ""

# Example usage
get_page(url)

driver.quit()

No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link found in this element
No link fo

In [19]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import requests
from PyPDF2 import PdfReader
import os
from docx import Document

def setup_driver():  
    """Configure and return a headless Chrome driver"""  
    options = webdriver.ChromeOptions()  
    options.add_argument("--headless")  
    options.add_argument("--disable-gpu")  
    options.add_argument("--no-sandbox")  
    options.add_argument("--disable-dev-shm-usage")  
    options.add_argument("--window-size=1920x1080")  
    return webdriver.Chrome(options=options)  

driver = setup_driver()
years = ['2025', '2024', '2023', '2022', '2021', '2020', '2019', '2018', '2017' , '2016',  '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004' ,'2003', '2002', '2001', '2000', '1999', '1998' ,'1997', '1996', '1995', '1994', '1993', '1992', '1991', '1990', '1989', '1988', '1987', '1986', '1985', '1984', '1983', '1982', '1981', '1980', '1979', '1978', '1977', '1976' ,'1975', '1974', '1972', '1971', '1970', '1969' , '1967', '1966', '1964', '1961' ,'1954', '1944', '1938', '1930']

url = 'https://new.kenyalaw.org/judgments/all/2025/'
data = []

def get_page(url):
    driver.get(url)
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="doc-table"]/table/tbody/tr')))
        articles = driver.find_elements(By.XPATH,'//*[@id="doc-table"]/table/tbody/tr')

        for article in articles:
            try:
                article_title = article.find_element(By.XPATH, './/td').text
                article_link = article.find_element(By.XPATH, './/td/a').get_attribute('href')
                content = get_content(article_link, article_title)  # Pass title to get_content

                data.append({
                    'title': article_title,
                    'link': article_link,
                    'content': content
                })
                df = pd.DataFrame(data)
                df.to_csv('judgements_pg1.csv', index=False)

            except NoSuchElementException:
                print('No link found in this element')               

    except TimeoutException:
        print("TimeoutException: Element not found")

def download_docx(docx_url, output_path):
    response = requests.get(docx_url)
    if response.status_code == 200:
        with open(output_path, 'wb') as file:
            file.write(response.content)
        print(f"DOCX downloaded to {output_path}")
    else:
        raise Exception(f"Failed to download DOCX. Status code: {response.status_code}")

def extract_docx_content(docx_path):
    content = ""
    doc = Document(docx_path)
    for para in doc.paragraphs:
        content += para.text + "\n"
    return content

def sanitize_filename(title):
    # Replace invalid characters and truncate
    invalid_chars = '<>:"/\\|?*'
    for char in invalid_chars:
        title = title.replace(char, '_')
    return title[:50] + '.docx'  # Adjust truncation length as needed

def get_content(url, article_title):
    try:
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'document-content__inner')))
        docx_element = driver.find_element(By.XPATH, '//a[contains(text(), "Download DOCX")]')
        docx_url = docx_element.get_attribute('href')
        
        # Generate safe filename
        filename = sanitize_filename(article_title)
        download_dir = 'downloaded_docs'
        os.makedirs(download_dir, exist_ok=True)
        docx_path = os.path.join(download_dir, filename)
        
        # Download and process DOCX
        download_docx(docx_url, docx_path)
        docx_content = extract_docx_content(docx_path)
        
        # Cleanup
        os.remove(docx_path)
        return docx_content
        
    except TimeoutException:
        print(f"TimeoutException: Element not found at {url}")
        return ""
    except Exception as e:
        print(f"Error processing {url}: {str(e)}")
        return ""

# Execute
get_page(url)
driver.quit()

No link found in this element
DOCX downloaded to downloaded_docs\Kithae v Tana Water Works Development Agency & ano.docx
Error processing https://new.kenyalaw.org/akn/ke/judgment/kelat/2025/158/eng@2025-03-10: Package not found at 'downloaded_docs\Kithae v Tana Water Works Development Agency & ano.docx'


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=134.0.6998.88); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF71D9EFE45+26629]
	(No symbol) [0x00007FF71D956010]
	(No symbol) [0x00007FF71D7E931A]
	(No symbol) [0x00007FF71D7FF424]
	(No symbol) [0x00007FF71D7FDEF3]
	(No symbol) [0x00007FF71D7F19A9]
	(No symbol) [0x00007FF71D7EF782]
	(No symbol) [0x00007FF71D7F3A3C]
	(No symbol) [0x00007FF71D7F3B0F]
	(No symbol) [0x00007FF71D83F1A0]
	(No symbol) [0x00007FF71D83FB1C]
	(No symbol) [0x00007FF71D83228C]
	(No symbol) [0x00007FF71D867AEF]
	(No symbol) [0x00007FF71D832156]
	(No symbol) [0x00007FF71D867CC0]
	(No symbol) [0x00007FF71D890169]
	(No symbol) [0x00007FF71D867883]
	(No symbol) [0x00007FF71D830550]
	(No symbol) [0x00007FF71D831803]
	GetHandleVerifier [0x00007FF71DD472BD+3529853]

In [2]:
import pandas as pd
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from docx import Document
import uuid
import re

def sanitize_filename(title):
    # Remove special characters and limit length
    clean_title = re.sub(r'[\\/*?:"<>|&]', '', title)[:100]
    return f"{clean_title}_{str(uuid.uuid4())[:8]}.docx"

def get_page(url):
    driver.get(url)
    try:
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '//*[@id="doc-table"]/table')))
        
        # Collect all links first before processing
        articles = driver.find_elements(By.XPATH, '//*[@id="doc-table"]/table/tbody/tr')
        links = []
        
        for article in articles:
            try:
                title = article.find_element(By.XPATH, './/td').text
                link = article.find_element(By.XPATH, './/td/a').get_attribute('href')
                links.append((title, link))
            except NoSuchElementException:
                print('Skipping article with missing elements')
                continue
                
        # Process collected links
        for title, link in links:
            try:
                content = get_content(link, title)
                data.append({'title': title, 'link': link, 'content': content})
                
                # Save incrementally
                pd.DataFrame(data).to_csv('judgements.csv', index=False)
                
            except Exception as e:
                print(f"Failed to process {link}: {str(e)}")

    except TimeoutException:
        print("Timeout loading initial page")

def get_content(url, title):
    try:
        # Use new tab to avoid DOM changes
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[1])
        driver.get(url)
        
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'document-content__inner')))
        
        # Get DOCX link
        docx_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, '//a[contains(text(), "Download DOCX")]'))
        )
        docx_url = docx_element.get_attribute('href')
        
        # Generate safe filename
        filename = sanitize_filename(title)
        download_path = os.path.join('downloads', filename)
        os.makedirs('downloads', exist_ok=True)
        
        # Download with stream to handle large files
        with requests.get(docx_url, stream=True) as r:
            r.raise_for_status()
            with open(download_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        
        # Read content
        doc = Document(download_path)
        content = '\n'.join([para.text for para in doc.paragraphs])
        
        # Clean up
        os.remove(download_path)
        
        return content
        
    finally:
        # Close tab and switch back
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

# Initialize driver
driver = setup_driver()
data = []

try:
    url = 'https://new.kenyalaw.org/judgments/all/2025/'

    get_page(url)
finally:
    driver.quit()

Skipping article with missing elements
Failed to process https://new.kenyalaw.org/akn/ke/judgment/kehc/2025/2975/eng@2025-03-17: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7F0B6FE45+26629]
	(No symbol) [0x00007FF7F0AD6010]
	(No symbol) [0x00007FF7F096931A]
	(No symbol) [0x00007FF7F09BF8E7]
	(No symbol) [0x00007FF7F09BFB1C]
	(No symbol) [0x00007FF7F0A134A7]
	(No symbol) [0x00007FF7F09E7AEF]
	(No symbol) [0x00007FF7F0A10169]
	(No symbol) [0x00007FF7F09E7883]
	(No symbol) [0x00007FF7F09B0550]
	(No symbol) [0x00007FF7F09B1803]
	GetHandleVerifier [0x00007FF7F0EC72BD+3529853]
	GetHandleVerifier [0x00007FF7F0EDDA22+3621858]
	GetHandleVerifier [0x00007FF7F0ED24D3+3575443]
	GetHandleVerifier [0x00007FF7F0C3B77A+860474]
	(No symbol) [0x00007FF7F0AE088F]
	(No symbol) [0x00007FF7F0ADCBC4]
	(No symbol) [0x00007FF7F0ADCD66]
	(No symbol) [0x00007FF7F0ACC2C9]
	BaseThreadInitThunk [0x00007FF9980E259D+29]
	RtlUserThreadStart [0x00007FF99A26AF38+40]

Failed to process https://new.kenyalaw.org/akn

In [1]:
import pandas as pd
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from docx import Document
import uuid
import re
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    return webdriver.Chrome(options=options)

def sanitize_filename(title):
    # Stricter filename cleaning
    clean_title = re.sub(r'[^a-zA-Z0-9 \-\_\(\)]', '', title)
    clean_title = re.sub(r'\s+', '_', clean_title)[:50]
    return f"{clean_title}_{uuid.uuid4().hex[:6]}.docx"

def requests_session():
    """Configure retry strategy"""
    session = requests.Session()
    retries = Retry(
        total=3,
        backoff_factor=0.5,
        status_forcelist=[500, 502, 503, 504]
    )
    session.mount('https://', HTTPAdapter(max_retries=retries))
    return session

def get_page(url):
    driver.get(url)
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="doc-table"]/table')))
    
    # Get all links first
    articles = driver.find_elements(By.XPATH, '//*[@id="doc-table"]/table/tbody/tr')
    links = []
    
    for article in articles:
        try:
            title = article.find_element(By.XPATH, './/td').text
            link = article.find_element(By.XPATH, './/td/a').get_attribute('href')
            links.append((title, link))
        except NoSuchElementException:
            print('Skipping article with missing elements')
            continue
    
    # Process links with rate limiting
    session = requests_session()
    download_dir = 'valid_docs'
    os.makedirs(download_dir, exist_ok=True)
    
    for idx, (title, link) in enumerate(links):
        try:
            print(f"Processing {idx+1}/{len(links)}: {title}")
            content = get_content(session, link, title, download_dir)
            
            if content:
                data.append({'title': title, 'link': link, 'content': content})
                pd.DataFrame(data).to_csv('judgements.csv', index=False)
            
            # Rate limiting delay
            time.sleep(2)
            
        except Exception as e:
            print(f"Failed to process {link}: {str(e)}")

def get_content(session, url, title, download_dir):
    try:
        # Get DOCX URL
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[1])
        driver.get(url)
        
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, '//a[contains(text(), "Download DOCX")]'))
        )
        docx_url = driver.find_element(By.XPATH, '//a[contains(text(), "Download DOCX")]').get_attribute('href')
        
        # Validate DOCX URL
        if not docx_url.endswith('.docx'):
            print(f"Invalid DOCX URL: {docx_url}")
            return ""
            
        # Generate safe filename
        filename = sanitize_filename(title)
        filepath = os.path.join(download_dir, filename)
        
        # Download with validation
        response = session.get(docx_url, stream=True, timeout=15)
        response.raise_for_status()
        
        # Verify content type
        if 'application/vnd.openxmlformats-officedocument.wordprocessingml.document' not in response.headers.get('Content-Type', ''):
            print(f"Invalid content type: {response.headers.get('Content-Type')}")
            return ""
        
        # Save file
        with open(filepath, 'wb') as f:
            for chunk in response.iter_content(8192):
                f.write(chunk)
        
        # Validate DOCX structure
        try:
            doc = Document(filepath)
            content = '\n'.join(para.text for para in doc.paragraphs if para.text.strip())
            
            # Keep valid files for reference
            return content
            
        except Exception as e:
            print(f"Invalid DOCX file: {str(e)}")
            os.remove(filepath)
            return ""
            
    except Exception as e:
        print(f"Download failed: {str(e)}")
        return ""
        
    finally:
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

# Initialize
driver = setup_driver()
data = []

try:
    get_page(url)
finally:
    driver.quit()

NameError: name 'url' is not defined

In [1]:
import pandas as pd
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from docx import Document
import uuid
import re
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from selenium.common.exceptions import NoSuchElementException, TimeoutException

# Configuration
years = ['2025']#, '2024', '2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000', '1999', '1998', '1997', '1996', '1995', '1994', '1993', '1992', '1991', '1990', '1989', '1988', '1987', '1986', '1985', '1984', '1983', '1982', '1981', '1980', '1979', '1978', '1977', '1976', '1975', '1974', '1972', '1971', '1970', '1969', '1967', '1966', '1964', '1961', '1954', '1944', '1938', '1930']
BASE_URL = "https://new.kenyalaw.org/judgments/all/{year}/?page={page}"
DOWNLOAD_DIR = 'valid_docs'
CSV_FILE = 'judgements_v2.csv'

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    return webdriver.Chrome(options=options)

def sanitize_filename(title):
    clean_title = re.sub(r'[^a-zA-Z0-9 \-\_\(\)]', '', title)
    clean_title = re.sub(r'\s+', '_', clean_title)[:50]
    return f"{clean_title}_{uuid.uuid4().hex[:6]}.docx"

def requests_session():
    session = requests.Session()
    retries = Retry(
        total=3,
        backoff_factor=0.5,
        status_forcelist=[500, 502, 503, 504]
    )
    session.mount('https://', HTTPAdapter(max_retries=retries))
    return session

def process_judgment(session, link, title, driver):
    try:
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[1])
        driver.get(link)
        
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, '//a[contains(text(), "Download DOCX")]'))
        )
        docx_url = driver.find_element(By.XPATH, '//a[contains(text(), "Download DOCX")]').get_attribute('href')
        
        if not docx_url.endswith('.docx'):
            return None

        filename = sanitize_filename(title)
        filepath = os.path.join(DOWNLOAD_DIR, filename)
        
        response = session.get(docx_url, stream=True, timeout=15)
        response.raise_for_status()

        if 'vnd.openxmlformats-officedocument.wordprocessingml.document' not in response.headers.get('Content-Type', ''):
            return None

        with open(filepath, 'wb') as f:
            for chunk in response.iter_content(8192):
                f.write(chunk)

        try:
            doc = Document(filepath)
            return '\n'.join(para.text for para in doc.paragraphs if para.text.strip())
        except Exception:
            os.remove(filepath)
            return None
            
    except Exception as e:
        print(f"Error processing {title}: {str(e)}")
        return None
    finally:
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

def scrape_judgments():
    driver = setup_driver()
    session = requests_session()
    os.makedirs(DOWNLOAD_DIR, exist_ok=True)
    data = []

    try:
        for year in years:
            page = 1
            while True:
                current_url = BASE_URL.format(year=year, page=page)
                print(f"Processing: {current_url}")
                
                try:
                    driver.get(current_url)
                    WebDriverWait(driver, 20).until(
                        EC.presence_of_element_located((By.XPATH, '//*[@id="doc-table"]/table'))
                    )
                except TimeoutException:
                    print(f"No data found for {year} page {page}")
                    break

                judgments = driver.find_elements(By.XPATH, '//*[@id="doc-table"]/table/tbody/tr')
                if not judgments:
                    break

                for judgment in judgments:
                    try:
                        title = judgment.find_element(By.XPATH, './/td').text
                        link = judgment.find_element(By.XPATH, './/td/a').get_attribute('href')
                    except NoSuchElementException:
                        continue

                    content = process_judgment(session, link, title, driver)
                    if content:
                        data.append({'title': title, 'link': link, 'content': content})
                        pd.DataFrame(data).to_csv(CSV_FILE, index=False)
                    time.sleep(1)

                try:
                    next_button = driver.find_element(By.XPATH, '//a[contains(text(), "Next")]')
                    if 'disabled' in next_button.get_attribute('class'):
                        break
                    page += 1
                except NoSuchElementException:
                    break
    finally:
        driver.quit()
        print("Scraping completed. Driver closed.")

if __name__ == "__main__":
    scrape_judgments()

Processing: https://new.kenyalaw.org/judgments/all/2025/?page=1
Error processing Bosire & 4 others v Superintendant, Rift Valley Provincial General Hospital Nakuru & another (Judicial Review E010 of 2024) [2025] KEELRC 765 (KLR) (14 March 2025) (Ruling)
Labour Law: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF61AC4FE45+26629]
	(No symbol) [0x00007FF61ABB6010]
	(No symbol) [0x00007FF61AA4931A]
	(No symbol) [0x00007FF61AA9F8E7]
	(No symbol) [0x00007FF61AA9FB1C]
	(No symbol) [0x00007FF61AAF34A7]
	(No symbol) [0x00007FF61AAC7AEF]
	(No symbol) [0x00007FF61AAF0169]
	(No symbol) [0x00007FF61AAC7883]
	(No symbol) [0x00007FF61AA90550]
	(No symbol) [0x00007FF61AA91803]
	GetHandleVerifier [0x00007FF61AFA72BD+3529853]
	GetHandleVerifier [0x00007FF61AFBDA22+3621858]
	GetHandleVerifier [0x00007FF61AFB24D3+3575443]
	GetHandleVerifier [0x00007FF61AD1B77A+860474]
	(No symbol) [0x00007FF61ABC088F]
	(No symbol) [0x00007FF61ABBCBC4]
	(No symbol) [0x00007FF61ABBCD66]
	(No symbol) [0x00007FF61ABAC2C9]

KeyboardInterrupt: 

In [2]:
import pandas as pd
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from docx import Document
import uuid
import re
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from selenium.common.exceptions import NoSuchElementException, TimeoutException, WebDriverException

# Configuration
years = ['2025', '2024', '2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000', '1999', '1998', '1997', '1996', '1995', '1994', '1993', '1992', '1991', '1990', '1989', '1988', '1987', '1986', '1985', '1984', '1983', '1982', '1981', '1980', '1979', '1978', '1977', '1976', '1975', '1974', '1972', '1971', '1970', '1969', '1967', '1966', '1964', '1961', '1954', '1944', '1938', '1930']
BASE_URL = "https://new.kenyalaw.org/judgments/all/{year}/?page={page}"
DOWNLOAD_DIR = 'valid_docs'
CSV_FILE = 'judgements_v2.csv'
MAX_RETRIES = 3

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    return webdriver.Chrome(options=options)

def sanitize_filename(title):
    clean_title = re.sub(r'[^a-zA-Z0-9 \-\_\(\)]', '', title)
    clean_title = re.sub(r'\s+', '_', clean_title)[:50]
    return f"{clean_title}_{uuid.uuid4().hex[:6]}.docx"

def requests_session():
    session = requests.Session()
    retries = Retry(
        total=3,
        backoff_factor=0.5,
        status_forcelist=[500, 502, 503, 504]
    )
    session.mount('https://', HTTPAdapter(max_retries=retries))
    return session

def process_judgment(session, link, title, driver):
    original_url = driver.current_url
    try:
        for attempt in range(MAX_RETRIES):
            try:
                driver.get(link)
                WebDriverWait(driver, 30).until(
                    EC.element_to_be_clickable((By.XPATH, '//a[contains(text(), "Download DOCX")]'))
                )
                docx_element = driver.find_element(By.XPATH, '//a[contains(text(), "Download DOCX")]')
                docx_url = docx_element.get_attribute('href')
                
                if not docx_url or not docx_url.endswith('.docx'):
                    print(f"No valid DOCX link for {title}")
                    return None

                filename = sanitize_filename(title)
                filepath = os.path.join(DOWNLOAD_DIR, filename)
                
                response = session.get(docx_url, stream=True, timeout=30)
                print(f"Downloading {docx_url} (Status: {response.status_code})")
                response.raise_for_status()

                if 'vnd.openxmlformats-officedocument.wordprocessingml.document' not in response.headers.get('Content-Type', ''):
                    print(f"Invalid content type for {title}")
                    return None

                with open(filepath, 'wb') as f:
                    for chunk in response.iter_content(8192):
                        f.write(chunk)

                try:
                    doc = Document(filepath)
                    content = '\n'.join(para.text for para in doc.paragraphs if para.text.strip())
                    return content
                except Exception as e:
                    print(f"Failed to read DOCX for {title}: {e}")
                    os.remove(filepath)
                    return None
                break
            except (TimeoutException, WebDriverException) as e:
                if attempt < MAX_RETRIES - 1:
                    print(f"Retrying {title} (Attempt {attempt + 1})")
                    time.sleep(2)
                    continue
                else:
                    raise
    except Exception as e:
        print(f"Error processing {title}: {str(e)} [URL: {link}]")
        return None
    finally:
        driver.get(original_url)
        time.sleep(1)

def scrape_judgments():
    driver = setup_driver()
    session = requests_session()
    os.makedirs(DOWNLOAD_DIR, exist_ok=True)
    data = []

    try:
        for year in years:
            page = 1
            while True:
                current_url = BASE_URL.format(year=year, page=page)
                print(f"Processing: {current_url}")
                
                try:
                    driver.get(current_url)
                    WebDriverWait(driver, 30).until(
                        EC.presence_of_element_located((By.XPATH, '//*[@id="doc-table"]/table'))
                    )
                except TimeoutException:
                    print(f"No data found for {year} page {page}")
                    break

                judgments = driver.find_elements(By.XPATH, '//*[@id="doc-table"]/table/tbody/tr')
                if not judgments:
                    break

                for judgment in judgments:
                    try:
                        title = judgment.find_element(By.XPATH, './/td').text.strip()
                        link = judgment.find_element(By.XPATH, './/td/a').get_attribute('href')
                    except NoSuchElementException:
                        continue

                    content = process_judgment(session, link, title, driver)
                    if content:
                        data.append({'title': title, 'link': link, 'content': content})
                        pd.DataFrame(data).to_csv(CSV_FILE, index=False)
                    time.sleep(1)

                try:
                    next_button = driver.find_element(By.XPATH, '//a[contains(text(), "Next")]')
                    if 'disabled' in next_button.get_attribute('class'):
                        break
                    page += 1
                except NoSuchElementException:
                    break
    finally:
        driver.quit()
        print("Scraping completed. Driver closed.")

if __name__ == "__main__":
    scrape_judgments()

Processing: https://new.kenyalaw.org/judgments/all/2025/?page=1
No valid DOCX link for A Thuo Kanai Advocates v Cannon Assurance Limited (Miscellaneous Application 125 of 2013) [2025] KEHC 2786 (KLR) (Commercial and Tax) (14 March 2025) (Ruling)
Scraping completed. Driver closed.


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=134.0.6998.89); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7165CFE45+26629]
	(No symbol) [0x00007FF716536010]
	(No symbol) [0x00007FF7163C931A]
	(No symbol) [0x00007FF7163DF424]
	(No symbol) [0x00007FF7163DDEF3]
	(No symbol) [0x00007FF7163D19A9]
	(No symbol) [0x00007FF7163CF782]
	(No symbol) [0x00007FF7163D3A3C]
	(No symbol) [0x00007FF7163D3B0F]
	(No symbol) [0x00007FF71641F1A0]
	(No symbol) [0x00007FF71641FB1C]
	(No symbol) [0x00007FF71641228C]
	(No symbol) [0x00007FF716447AEF]
	(No symbol) [0x00007FF716412156]
	(No symbol) [0x00007FF716447CC0]
	(No symbol) [0x00007FF716470169]
	(No symbol) [0x00007FF716447883]
	(No symbol) [0x00007FF716410550]
	(No symbol) [0x00007FF716411803]
	GetHandleVerifier [0x00007FF7169272BD+3529853]